In [1]:
CPU_CORES = 51
addprocs(CPU_CORES - 1);
@everywhere using DistributedArrays, JuMP, Distributions, Ipopt
using StatsBase

In [2]:
print(nprocs(),"\n")
print(nworkers(),"\n")
print(workers(),"\n")

51
50
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [3]:
@everywhere landscape1= [0. 0 0 2 2 0 0 0 2 0 0 2;
              2 0 1 2 5 5 0 0 2 0 0 0;
              3 1 2 2 1 1 2 0 0 3 0 0;
              2 1 1 2 1 2 2 0 3 4 4 0
              2 2 1 1 1 1 1 0 3 4 4 3;
              2 2 1 1 1 1 1 1 3 3 3 0;
              0 2 0 1 5 5 5 0 1 1 0 3;
              3 0 0 1 0 0 2 0 0 3 1 3;
              3 4 0 1 1 3 2 2 2 3 3 3;
              4 3 2 0 0 3 2 2 2 3 2 2] + 1
@everywhere Ncell = 30

In [4]:
@everywhere function create_pixel_mat(landscape, ordered_vec)
    retmat = deepcopy(landscape)
    for i = 1:length(ordered_vec)
        retmat[abs.(retmat - i) .< 0.000000001] = ordered_vec[i]
    end
    
    if size(retmat) == (10,12)
        retmat_zone = hcat(reshape(retmat[1:5 , 1:6],Ncell),
            reshape(retmat[1:5 , 7:12],Ncell),
            reshape(retmat[6:10 , 1:6],Ncell),
            reshape(retmat[6:10 , 7:12],Ncell))
    end
    retmat_zone
end


In [5]:
# Using the prior and visibility info given in the DCA paper
# These have @everywhere because they are fixed. Better declare them in each process rather than
# copying from the parent process
@everywhere s1 = create_pixel_mat(landscape1, [0.4,0.9,0.3,0.2,0.1,0.8])
@everywhere s2 = create_pixel_mat(landscape1, [0.5,0.1,0.1,0.7,0.6,0.9])
@everywhere s3 = create_pixel_mat(landscape1, [0.6,0.1,0.4,0.8,0.7,0.1])
@everywhere s4 = create_pixel_mat(landscape1, [0.8,0.1,0.6,0.2,0.1,0.7])
@everywhere s5 = create_pixel_mat(landscape1, [0.5,0.3,0.5,0.4,0.3,0.6])
@everywhere s6 = create_pixel_mat(landscape1, [0.1,0.5,0.2,0.6,0.5,0.2])




@everywhere Z = 4 # number of zones
@everywhere S = 6 # number of sensors
W = reshape(hcat(s1,s2,s3,s4,s5,s6),Ncell,Z,S) # visibility tensor

# Decrease censors' visibility
W = W/6



@everywhere θ = 0.3
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
@everywhere epsilon = 1e-5


# No @everywhere because these are only used by the parent process
maxiter = 15 # number of iterations for 



# No @everywhere because these are changing, pass into functions as parameters

Φ = 5
# prior_ordered_vec = [0.0085,0.001,0.0115,0.013,0.014,0] # prior of target, order given in the paper
# prior = create_pixel_mat(landscape1, prior_ordered_vec)
# print(sum(prior))





5

In [6]:



####################################
## CE Algorithm parallel N

@everywhere function solve_alloc(u, Φ,prior,W)

    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S]) # resource allocation for each senor in each zone in each cell
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    getobjectivevalue(m_ϕ), transpose(u)
end
    
    

function CE_parallel(Φ,prior,W)
    
    P = ones(S,Z)/Z
    ind = true
    P_holder = zeros(S,Z)
    for iter=1:maxiter

        cumsumP = cumsum(P,2)
        
        vec_tile = rand(Uniform(0,1), S * N,1)
        cumsumP_tile = repmat(cumsumP,N)
        which_zone = floor.(findmax(cumsumP_tile .> vec_tile,2)[2] / (S*N) - epsilon) + 1
        u = abs.(repmat(transpose(1:Z),S*N,1) .- which_zone) .< epsilon
        u = transpose(u)
        
        arr_obj = []
        arr_assign = zeros(S,Z,N)
#         tic()
        retobj = @DArray [solve_alloc(u[:,(i-1)*S+1:i*S],Φ,prior,W) for i = 1:N]
#         toc()
        for i = 1:N
            push!(arr_obj, retobj[i][1])
            arr_assign[:,:,i] = retobj[i][2]
        end
 
        best_ind = sortperm(arr_obj)[1:Threshold]
        P = squeeze(sum(arr_assign[:,:,best_ind],3)/Threshold,3)
        
        if sum(abs.(P - 1) .<=0.000001) == S
#             print(iter,'\n')
            break
        end
    end
    
    u = transpose(P)
    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S])
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    
    getobjectivevalue(m_ϕ), P, exp.(getvalue(aux)), getvalue(ϕvar)
    
    
    
end


CE_parallel (generic function with 1 method)

In [8]:
function map2repr(x,y)
    xoffset = [0,6,0,6]
    yoffset = [0,0,5,5]
    which_zone = 1 + (x > centerx)  + (y > centery) * 2
    xoff = x - xoffset[which_zone]
    yoff = y - yoffset[which_zone]
    which_cell = (xoff-1) * 5 + yoff
    return which_cell, which_zone
end


# Make prior at the centre of the landscape
# raidus 1, 50% 
# radius 2, 50%



prior = zeros(30,4)
radius = 2
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 12
    end
end


radius = 1
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 4
    end
end

In [9]:
# Test run. good practice to just run it when first open the notebook (initialize the optimizer for each process)

tic()
Φ = 5
maxiter = 20
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,prior,W)
toc()
print(obj,'\n')

print(assignment,'\n')

	From worker 20:	
	From worker 20:	******************************************************************************
	From worker 20:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 20:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 20:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 20:	******************************************************************************
	From worker 20:	
	From worker 10:	
	From worker 10:	******************************************************************************
	From worker 10:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 10:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 10:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 10:	******************************************************************************
	Fro

	From worker 8:	
	From worker 8:	******************************************************************************
	From worker 8:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 8:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 8:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 8:	******************************************************************************
	From worker 8:	
	From worker 25:	
	From worker 25:	******************************************************************************
	From worker 25:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 25:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 25:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 25:	******************************************************************************
	From worke

In [ ]:
##############################################
## Moving Target

In [10]:
# Assume Ncell,Z exist, current ONLY WORK WITH THE LANDSCAPE IN PAPERS
# construct a 30x4x30x4 one step moving probability tensor

function build_c2c_tensor(matrix)
    

    
    # first, construct 120x120 one step transition matrix

    # here we number all cells from 1 to 120 in a row-major fashion
    c2c = zeros(Ncell * Z, Ncell * Z)
    for i = 1:(Ncell * Z)
        # Stay in current cell
        c2c[i,i] = matrix[2,2]

        # move right
        if mod(i,12) ==  0 # right end 
            c2c[i,i] = c2c[i,i] + matrix[2,3]
        else
            c2c[i,i+1] = matrix[2,3]
        end


        # move left
        if mod(i,12) ==  1 # left end 
            c2c[i,i] = c2c[i,i] + matrix[2,1]
        else
            c2c[i,i-1] = matrix[2,1]
        end

        # move down
        if i > 12 * (10-1) # bottom end
            c2c[i,i] = c2c[i,i] + matrix[3,2]
        else
            c2c[i,i+12] = matrix[3,2]
        end

        # move up
        if i <= 12 # top row
            c2c[i,i] = c2c[i,i] + matrix[1,2]
        else
            c2c[i,i-12] = matrix[1,2]
        end



        # move diagonal bottom-right
        if (mod(i,12) == 0) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,3]
        else
            c2c[i,i+12+1] = matrix[3,3]
        end


        # move diagonal top-right
        if (mod(i,12) == 0) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,3]
        else
            c2c[i,i-12+1] = matrix[1,3]
        end

        # move diagonal top-left
        if (mod(i,12) == 1) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,1]
        else
            c2c[i,i-12-1] = matrix[1,1]
        end

        # move diagonal bottom-left
        if (mod(i,12) == 1) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,1]
        else
            c2c[i,i+12-1] = matrix[3,1]
        end


    end

    # convert the 120x120 matrix into 30x4x30x4 tensor
    # Note that Julia is column major: suppose the first (leftmost, upmost) cell
    # is cell 1, cell 2 refers to the cell below cell 1 not the cell to the right
    # This affects how cell index is assigned
    c2c_tensor = zeros(Ncell,Z,Ncell,Z)
    for i=1:Ncell*Z
        zone_prev = (i / 60 > 1) * 2 + ((mod(i,12) / 6 > 1) | (mod(i,12) == 0)) + 1
        offset_prev = [0,6,60,66][zone_prev]
        cell_prev = (rem(i-offset_prev,12) - 1) * 5 + div(i-offset_prev,12) + 1 # column major in julia
        for j=1:Ncell*Z
            zone_cur = (j / 60 > 1) * 2 + ((mod(j,12) / 6 > 1) | (mod(j,12) == 0)) + 1
            offset_cur = [0,6,60,66][zone_cur]
            cell_cur = (rem(j-offset_cur,12) - 1) * 5 + div(j-offset_cur,12) + 1 # column major in julia
            c2c_tensor[cell_prev,zone_prev,cell_cur,zone_cur] = c2c[i,j]
        end
    end
    
    return c2c_tensor
end

build_c2c_tensor (generic function with 1 method)

In [11]:
# Myopic and Optimal Search Plan (without cross-cueing)

In [12]:
T = 4
Φ = 5
Total_Iter = 1 # the number of iterations to go from myopic search plan to optimal search plan

1

In [307]:


function mov_target_CE(prior,W,c2c_tensor)
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior


        for τ = 1:T
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β,W)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end




function compute_non_detection(prior, W, c2c_tensor,final_assignment,final_res_alloc)
    # Compute nondetection probability at each time step (used in computing U and D)
    nondetection_prob_holder = zeros(Ncell,Z,T)
    for i=1:T
        res_alloc_cur = final_res_alloc[:,:,:,i]
        assign_cur = final_assignment[:,:,i]
        nondetection_prob_holder[:,:,i] = exp.( squeeze(-sum(W .* res_alloc_cur .* reshape(transpose(assign_cur),1,Z,S),3),3))
    end


    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(nondetection_prob_holder[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end

## For optimal solution, also need to incorporate DTensor, for myopic solution, do not incorporate DTensor
#     DTensor_holder = ones(Ncell,Z,T)
#     for i = T-1:-1:1
#         DTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
#                 reshape(nondetection_prob_holder[:,:,i+1],1,1,Ncell,Z) .* 
#                 reshape(DTensor_holder[:,:,i+1],1,1,Ncell,Z),3),4)
#     end
    
#     β_holder = UTensor_holder .* DTensor_holder
    
    β_holder = UTensor_holder
    return sum(sum(β_holder[:,:,:] .* nondetection_prob_holder[:,:,:],1),2)
end 
     
    
function compute_non_detection_test(prior, W, c2c_tensor,prob_nondetection_tensor,T)
    # Compute nondetection probability at each time step (used in computing U and D)



    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(prob_nondetection_tensor[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end


    β_holder = UTensor_holder
    return sum(sum(β_holder[:,:,:] .* prob_nondetection_tensor[:,:,τ],1),2)
end 




compute_non_detection_test (generic function with 2 methods)

In [297]:
# Modify the true moving distribution so I can meaningfully simulate longer time step with 
# target probably at the center of the landscape (10x12)
moving_mat= [0.0 0.0 0.0;
             0.0 0.5 0.125;
             0.0 0.125 0.25]
c2c_tensor_true = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")

moving_mat= ones(3,3)/9
c2c_tensor_random = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")


moving_mat= [0.25 0.125 0.0;
             0.125 0.5 0.0;
             0.0 0.0 0.0]
c2c_tensor_opposite = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")


moving_mat= [0.0 0.125 0.25;
             0.0 0.5 0.125;
             0.0 0.0 0.0]
c2c_tensor_opposite = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")

1.0
1.0000000000000002
1.0
1.0


In [288]:
# When you have the true moving probability

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor)
print(nondetection_no_movement_test,"\n")
toc()


Iter: 1. Completed timestep: 1. Obj: 0.7968924057097808
Iter: 1. Completed timestep: 2. Obj: 0.6432800530034897
Iter: 1. Completed timestep: 3. Obj: 0.5257650588703162
Iter: 1. Completed timestep: 4. Obj: 0.4487002847192498
Iter: 1. Completed timestep: 5. Obj: 0.3924907702342038
Iter: 1. Completed timestep: 6. Obj: 0.35039078434541154
Iter: 1. Completed timestep: 7. Obj: 0.3152900834966408
Iter: 1. Completed timestep: 8. Obj: 0.2842794495686143
Iter: 1. Completed timestep: 9. Obj: 0.2550584327303841
[0.796892]

[0.64328]

[0.525765]

[0.4487]

[0.392491]

[0.350391]

[0.31529]

[0.284279]

[0.255058]
elapsed time: 38.807830032 seconds


38.807830032

In [298]:
# When you DO NOT have the true moving probability and we assume uniform moving probability

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_random)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor)
print(nondetection_no_movement_test,"\n")
toc()


# Most of the time, actual ND > expected ND, but not always. because, we have a prior, then compute a allocation
# => ND tensor (minimize the expected ND). But given this ND tensor, starting prior might not be the
# expected ND minimizer associated with this ND tensor

Iter: 1. Completed timestep: 1. Obj: 0.7968924057448765
Iter: 1. Completed timestep: 2. Obj: 0.6823453321263124
Iter: 1. Completed timestep: 3. Obj: 0.6035755076493168
Iter: 1. Completed timestep: 4. Obj: 0.5471959848071752
Iter: 1. Completed timestep: 5. Obj: 0.504092010006704
Iter: 1. Completed timestep: 6. Obj: 0.47037606870325444
Iter: 1. Completed timestep: 7. Obj: 0.4436198760509048
Iter: 1. Completed timestep: 8. Obj: 0.4214874525311414
Iter: 1. Completed timestep: 9. Obj: 0.40214130393034786
[0.796892]

[0.65919]

[0.575748]

[0.520449]

[0.47798]

[0.45896]

[0.452508]

[0.444049]

[0.436282]
elapsed time: 32.300996722 seconds


32.300996722

In [299]:
# What if I assume a distribution that is the exact opposite of the true distribution

T = 9

tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_opposite)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor)
print(nondetection_no_movement_test,"\n")
toc()

# expected ND should not mirror those calculated using true moving distribution because the landscape in
# opposite direction is not mirrored

Iter: 1. Completed timestep: 1. Obj: 0.7968924057488289
Iter: 1. Completed timestep: 2. Obj: 0.654855693350739
Iter: 1. Completed timestep: 3. Obj: 0.5490907743128995
Iter: 1. Completed timestep: 4. Obj: 0.47329763149502074
Iter: 1. Completed timestep: 5. Obj: 0.4192955404211606
Iter: 1. Completed timestep: 6. Obj: 0.3772088465913189
Iter: 1. Completed timestep: 7. Obj: 0.34213384214853826
Iter: 1. Completed timestep: 8. Obj: 0.31026548931103554
Iter: 1. Completed timestep: 9. Obj: 0.28169363786939866
[0.796892]

[0.683651]

[0.628776]

[0.595423]

[0.573483]

[0.568256]

[0.565831]

[0.565342]

[0.565082]
elapsed time: 36.785270004 seconds


36.785270004

In [291]:
# What if I assume a distribution (which makes detection easier), but the true distribution is uniform (completely random)

T = 9
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_random,prob_nondetect_Tensor)
print(nondetection_no_movement_test,"\n")
toc()


Iter: 1. Completed timestep: 1. Obj: 0.796892405712199
Iter: 1. Completed timestep: 2. Obj: 0.6383499452042547
Iter: 1. Completed timestep: 3. Obj: 0.5268131909231523
Iter: 1. Completed timestep: 4. Obj: 0.44805159005605855
Iter: 1. Completed timestep: 5. Obj: 0.3903710204142543
Iter: 1. Completed timestep: 6. Obj: 0.34798193837371966
Iter: 1. Completed timestep: 7. Obj: 0.3148070122826466
Iter: 1. Completed timestep: 8. Obj: 0.28353174832748473
Iter: 1. Completed timestep: 9. Obj: 0.2542818562426677
[0.796892]

[0.686967]

[0.624033]

[0.580609]

[0.553271]

[0.535238]

[0.520093]

[0.511837]

[0.506924]
elapsed time: 37.614816749 seconds


37.614816749

10

30×4×30×4 Array{Float64,4}:
[:, :, 1, 1] =
 0.681686   0.0  0.0  0.0
 0.140974   0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.221053   0.0  0.0  0.0
 0.0356081  0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 ⋮                       
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0

[:, :, 2, 1] =
 0.084863   0.0  0.0  0.0
 0.519454   0.0  0.0  0.0
 0.140974   0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0886734  0.0  0.0  0.0
 0.221053   0.0  0.0  0.0
 0.0356081  0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0        0.0  0.0  0.0
 0.0 

In [24]:
sum(unnorm)

4.866906860766538

In [ ]:
# # Modify the true moving distribution so I can meaningfully simulate longer time step with 
# # target probably at the center of the landscape (10x12)
# moving_mat= [0.0 0.0 0.0;
#              0.0 0.5 0.125;
#              0.0 0.125 0.25]
# c2c_tensor_true = build_c2c_tensor(moving_mat)
# print(sum(moving_mat),"\n")



# moving_mat= [0.25 0.125 0.0;
#              0.125 0.5 0.0;
#              0.0 0.0 0.0]
# c2c_tensor_opposite = build_c2c_tensor(moving_mat)
# print(sum(moving_mat),"\n")

In [367]:
moving_mat= [0.0 0.125 0.25;
             0.0 0.5 0.125;
             0.0 0.0 0.0]
c2c_tensor_opposite = build_c2c_tensor(moving_mat)
print(sum(moving_mat),"\n")

1.0


In [368]:
UTensor = zeros(Ncell,Z,T)
UTensor[:,:,1] =  prior
        
UTensor_actual = zeros(Ncell,Z,T)
UTensor_actual[:,:,1] =  prior

prob_nondetect_Tensor = zeros(Ncell,Z,T)
actuals = []

0-element Array{Any,1}

In [369]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
push!(actuals,obj)
obj

0.7968924012393583


0.796892405784697

In [370]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.6837700073592009
0.6377990835659304
1.0720774378292415

In [371]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,τ)
# nondetection_no_movement_test[:,:,τ]

1×1×2 Array{Float64,3}:
[:, :, 1] =
 0.830225

[:, :, 2] =
 0.68377

In [372]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
    trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [277]:
summarystats(movement_obj.* 1.0)

Summary Stats:
Mean:           0.698564
Minimum:        0.665442
1st Quartile:   0.691321
Median:         0.698971
3rd Quartile:   0.705551
Maximum:        0.730847


In [278]:
Threshold_movement = 20

20

In [279]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.0894366  0.199482   0.200489
 0.0654952  0.130866   0.130996
 0.0535616  0.0517176  0.077956

In [280]:
best_ind = sortperm(movement_obj)[num_CE_movementmet-Threshold_movement+1:num_CE_movementmet]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.116563  0.0374105  0.0297942
 0.161383  0.0801904  0.0690973
 0.185228  0.173864   0.14647  

In [281]:
best_ind = sortperm(abs.(movement_obj - actual))[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.0998985  0.126229  0.087498 
 0.0976805  0.1324    0.0874759
 0.128689   0.122875  0.117254 

In [373]:
best_ind = sortperm((movement_obj - actual .<= 0) + (movement_obj - actual))[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

3×3 Array{Float64,2}:
 0.114402   0.101389   0.155646
 0.101838   0.135641   0.109456
 0.0903362  0.0858283  0.105463

In [374]:

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")

0.9999999999999999


In [375]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_opposite .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.5933064634723089
0.5528851043058908
1.0731098719274854

In [376]:
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,τ)
# nondetection_no_movement_test[:,:,τ]

1×1×3 Array{Float64,3}:
[:, :, 1] =
 0.826416

[:, :, 2] =
 0.664313

[:, :, 3] =
 0.593306

In [399]:
num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []

for i=1:num_CE_movementmet
    moving_mat = rand(Uniform(0,1), 3,3)
    moving_mat = moving_mat/sum(moving_mat)
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,τ)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test[:,:,2:τ],(τ-1),1)))/(τ-1)
    
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
end


In [402]:
best_ind = sortperm(movement_obj)[1:Threshold_movement]
MV = squeeze(sum(movement_assign[:,:,best_ind],3)/Threshold_movement,3)

c2c_tensor_opposite = build_c2c_tensor(MV)
print(sum(moving_mat),"\n")
display(MV)

3×3 Array{Float64,2}:
 0.107978   0.183761   0.151744 
 0.0887307  0.130536   0.146696 
 0.0588837  0.0702668  0.0614044

1.0


In [182]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.46031107472039134
0.4349920360793713
1.0582057521540467

In [183]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.39727919829142067
0.3740232082025717
1.0621779332908494

In [184]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.34351640129631034
0.3199735765180854
1.0735774029668799

In [185]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.29659107422266384
0.27243138113543675
1.0886817553342591

In [186]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.2521701507286998
0.22205637628887034
1.1356131940145453

In [187]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.20386537561576465
0.17716304375546843
1.150721794423178

In [ ]:

# When you have the true moving probability
T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_true = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_true,"\n")
toc()




# When you DO NOT have the true moving probability and we assume uniform moving probability

T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_init)
nondetection_no_movement = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_no_movement,"\n")
toc()


# What if I assume a distribution (which makes detection easier), but the true distribution is uniform
T = 4
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_reverse = compute_non_detection(prior, W, c2c_tensor_init,final_assignment, final_res_alloc)
print(nondetection_no_movement_reverse,"\n")
toc()


In [213]:
T = 10
moving_mat= ones(3,3)/9
c2c_tensor_init = build_c2c_tensor(moving_mat)

30×4×30×4 Array{Float64,4}:
[:, :, 1, 1] =
 0.666667  0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 ⋮                      
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0

[:, :, 2, 1] =
 0.111111  0.0  0.0  0.0
 0.444444  0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.111111  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  

In [217]:
UTensor = zeros(Ncell,Z,T)
UTensor[:,:,1] =  prior
        
UTensor_actual = zeros(Ncell,Z,T)
UTensor_actual[:,:,1] =  prior

prob_nondetect_Tensor = zeros(Ncell,Z,T)


30×4×10 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮                 
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮                 
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0

In [218]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
obj

0.7676209062409314


0.7676209107863863

In [219]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.6169094520634583
0.6421428806874405
0.9607043395124636

In [220]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.5390437518519293
0.5546616897647201
0.9718424073611147

In [221]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4937341726499716
0.495495339315114
0.9964456443372873

In [222]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.478719219997796
0.45127989006743185
1.0608033518317515

In [224]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4705223486349652
0.4169832762576089
1.1283962101738592

In [225]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4670007470056159
0.38983554922197666
1.197942947834397

In [226]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4667986394629345
0.3676633133134658
1.2696361659150555

In [227]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

0.4661287912462162
0.3486592986713505
1.3369177102762246

In [67]:
# Currently only support myopic, total_iteration = 1.
# Eg. when I plan for T = 2, I use the non-detection at T = 1 based on actual movement distribution, 
# rather than the fictious non-detection based on my guessed movement distribution

function mov_target_CE_adaptive(prior,W,c2c_tensor_true,c2c_tensor_init)
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior
        
        UTensor_actual = zeros(Ncell,Z,T)
        UTensor_actual[:,:,1] =  prior


        for τ = 1:4
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
                UTensor_actual[:,:,τ] = sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β,W)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            
            # Instead of using prob_nondetect, calculated with the guessed movement distribution, 
            # using actual outcome calculated with true movement distribution
            # 
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
#             prob_nondetect_Tensor[:,:,τ] = exp.(squeeze(-sum(W .* res_alloc .* reshape(assignment,1,Z,S),3),3))
            
            
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end


mov_target_CE_adaptive (generic function with 1 method)

In [69]:
# ADAPTIVE: When you DO NOT have the true moving probability and we assume uniform moving probability
tic()

moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE_adaptive(prior,W,c2c_tensor_true,c2c_tensor_init)
nondetection_no_movement = compute_non_detection(prior, W, c2c_tensor_true,final_assignment, final_res_alloc)
print(nondetection_no_movement,"\n")


toc()

LoadError: [91mMethodError: no method matching mov_target_CE_adaptive(::Array{Float64,2}, ::Array{Float64,3}, ::Array{Float64,4})[0m
Closest candidates are:
  mov_target_CE_adaptive(::Any, ::Any, ::Any, [91m::Any[39m) at In[67]:2[39m

In [14]:
# No prior Moving

num_trials = 1
T = 4
Total_Iter = 1
new_prior = ones(N,Z) .* 1/(N*Z)
nondetection_no_prior_moving = zeros(num_trials)
Φ = 5
tic()
for i=1:num_trials
    tic()
    moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(new_prior,W,c2c_tensor)
    nondetection_no_prior_moving[i] = compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)
    toc()
end
toc()
summarystats(nondetection_no_prior_moving)

Iter: 1. Completed timestep: 1. Obj: 0.8489888636584664
Iter: 1. Completed timestep: 2. Obj: 0.6967427359265222
Iter: 1. Completed timestep: 3. Obj: 0.5582142934072027
Iter: 1. Completed timestep: 4. Obj: 0.4396014595267237
elapsed time: 13.596354489 seconds
elapsed time: 13.679596262 seconds


Summary Stats:
Mean:           0.425093
Minimum:        0.425093
1st Quartile:   0.425093
Median:         0.425093
3rd Quartile:   0.425093
Maximum:        0.425093


build_c2c_tensor (generic function with 1 method)

In [35]:
moving_mat_no= [1/9 1/9 1/9;
                 1/9 1/9 1/9;
                 1/9 1/9 1/9]
c2c_tensor_no = build_c2c_tensor(moving_mat_no)
sum(moving_mat_no)



1.0000000000000002

In [ ]:
# moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_no)
# compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)



In [ ]:
# No moving assumption 

tic()
T = 4
Total_Iter = 2
num_trials = 50
nondetection_no_movement_moving = zeros(num_trials)
Φ = 5
for i=1:num_trials
    tic()
    moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_no)
    nondetection_no_movement_moving[i] = compute_non_detection(prior, W, c2c_tensor,final_assignment, final_res_alloc)
    toc()
end
toc()
summarystats(nondetection_no_movement_moving)